### Imports
Import packages and load model parameters (uncertainties, levers, etc.)

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pickle

import plotly.express as px
import plotly.graph_objects as go
from ema_workbench.analysis import (prim, dimensional_stacking)
from ema_workbench import ema_logging

In [ ]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
    Samplers,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation
from ema_workbench.analysis import parcoords


def sum_over(*args):
    return sum(args)

In [ ]:
# Enable logging
ema_logging.log_to_stderr(ema_logging.INFO)

**Problem formulation**
Important cell below, here you can choose the problem formulation number, between 0-5
each problem formulation has its own list of outcomes

In [ ]:
used_problem_formulation = 3

dike_model, planning_steps = get_model_for_problem_formulation(used_problem_formulation)

### Policy formulations
In the following cells the wanted policies can be adjusted

In [ ]:
# Create a function which sets everything to 0 by default
def get_0_dict():
    return {l.name: 0 for l in dike_model.levers}

# Creates a Policy object from a dict and a (optional) name
def create_policy(dict1, name=None):
    return Policy(f"Policy_{name}", **dict(get_0_dict(), **dict1))

In the following policies formulations:
- `RfR` says if the Room for River projects are enabled (projects `0` to `5`). The first number indicated the project, the second (after `RfR`) if in which time step the measure it taken (currently `0` to `2`). The value can be either `0` (not enabled) or `1` (enabled).
- `A.1` to `A.5` are the different dikes. `A.3` is the dike ring around Zutphen. The number after `DikeIncrease` is again the timestep, the value assigned to the thing the heightening in decimeters.
- Early Warning Systems can also be enabled, with `EWS_DaysToThreat`, which specifies the early warning time in days.

See the implementation in problem_formulation.py (starting from line 35) for more details.

In [ ]:
pol_list = []
#pol_list.append(create_policy({}, name="Zero"))

for dm in [0, 10]:
    for location in [2]:
        for switch in [0,1]:
            for ews_days in [0, 4]:
                pol_list.append(create_policy({"A.3_DikeIncrease 0": dm, f"{location}_RfR 0": switch, "EWS_DaysToThreat": ews_days},
                                            name=f"Dike_{dm}dm_RfR_{location}{switch}_EWS_{ews_days}d"))


In [ ]:
policy

### Run the model (or load the data)
In the next cell the model is ran (if `use_pickle1 = False`) and the new results data is saved, or, if `use_pickle1 = True`, the saved results data is loaded.

In [ ]:
# True, use results in pickle file; False, run MultiprocessingEvaluator
use_pickle1 = True

if use_pickle1:
    with open('data/formulation_results.pickle', 'rb') as filehandler:
        results = pickle.load(filehandler)

else:
    # pass the policies list to EMA workbench experiment runs
    n_scenarios = 15000
    with MultiprocessingEvaluator(dike_model, n_processes=10) as evaluator:
        results = evaluator.perform_experiments(n_scenarios, pol_list, uncertainty_sampling=Samplers.LHS)

    # Save results in Pickle file
    with open('data/policyresults.pickle', 'wb') as filehandler:
        pickle.dump(results, filehandler)

### Process the results
In this section a dataframe is generated with all the outcomes data, and aggegrated to mean, min, max and quantiles values for each policy.

In [ ]:
policy_dict = {
    'Policy_Dike_0dm_RfR_20_EWS_0d': 'Policy 0', 
    'Policy_Dike_0dm_RfR_20_EWS_4d': 'Policy 1: EWS',
    'Policy_Dike_0dm_RfR_21_EWS_0d': 'Policy 2: RfR', 
    'Policy_Dike_0dm_RfR_21_EWS_4d': 'Policy 3: RfR+EWS',
    'Policy_Dike_10dm_RfR_20_EWS_0d': 'Policy 4: Dike',
    'Policy_Dike_10dm_RfR_20_EWS_4d': 'Policy 5: Dike+EWS',
    'Policy_Dike_10dm_RfR_21_EWS_0d': 'Policy 6: Dike+RfR', 
    'Policy_Dike_10dm_RfR_21_EWS_4d': 'Policy 7: Dike+RfR+EWS'
    }

In [ ]:
# Create a dataframe from outcomes, and add the policy column to it
exp, out = results
df = pd.DataFrame(out)
a = df.columns[df.columns.str.contains('Costs')]
df['Total Costs'] = df[a].sum(axis=1)
a = df.columns[df.columns.str.contains('Deaths')]
df['Total Deaths'] = df[a].sum(axis=1)

exp['policy'] = exp.policy.map(policy_dict)

df["policy"] = pd.DataFrame(exp)["policy"]
df

In [ ]:
with open('data/policyresults.pickle', 'wb') as filehandler:
    pickle.dump(df, filehandler)

In [ ]:
# Define two functions to get the 5th and 95th quantiles
def q05(x):
    return x.quantile(0.05)

def q95(x):
    return x.quantile(0.95)

In [ ]:
runplot=True
if runplot==True:
    data=df.drop(['policy'], axis=1)


    sns_plot = sns.pairplot(df, hue='policy',  vars=data.columns )
    sns_plot.savefig('pairplot.png')
    #plt.show()

In [ ]:
df_melt = pd.melt(df, id_vars='policy')
# Create two subplots and unpack the output array immediately
f, axes = plt.subplots(5, 3, sharex=True, figsize=(30, 50))
unq_pol = df_melt.variable.unique()
count = 0
axes = axes.flatten()
for axs in axes:
        if count<14:
                print(axs)
                sns.boxplot(x="policy", y="value", data=df_melt.loc[df_melt.variable ==unq_pol[count]], ax=axs)
                axs.set_xticklabels(axs.get_xticklabels(),rotation=90)
                axs.set_title(unq_pol[count])
                count+=1
        else:
                pass
f.savefig('boxplot.png')
plt.show()

In [ ]:
df.rename({
       'A.1 Total Costs': '1.C', 'A.1_Expected Number of Deaths': '1.D',
       'A.2 Total Costs': '2.C', 'A.2_Expected Number of Deaths': '2.D' , 'A.3 Total Costs': '3.C',
       'A.3_Expected Number of Deaths': '3.D', 'A.4 Total Costs': '4.C',
       'A.4_Expected Number of Deaths': '4.D', 'A.5 Total Costs': '5.C',
       'A.5_Expected Number of Deaths': '5.D',  'RfR Total Costs': 'RfR', 'Expected Evacuation Costs':'Evac Costs'
       },
       axis=1, inplace=True)

In [ ]:
pol_list = df['policy'].unique()

count=0
for i in pol_list:
    
    df.loc[df.policy== i, 'policy'] = count
    count+=1
    
limits = parcoords.get_limits(df.iloc[:,:-1])
dimensionlist = []

for column in limits:
    lower=0
    upper=limits[column].iloc[1]
    #print(upper)
    if upper>0:
        dimensionlist.append(dict(range = [lower,upper],
                label = column, values = df[column].values, tickformat = "~g"))
    else:
        dimensionlist.append(dict(range = [lower,0.1],
                label = column, values = df[column].values, tickformat = "~g"))

fig = go.Figure(data=
    go.Parcoords(line = dict(color = df['policy'], showscale=True),
                   dimensions= dimensionlist
                             ))
plt.savefig('explorationparallelplot.png')
fig.show()

In [ ]:
exp_select = exp.loc[exp.policy.isin(['Policy 5: Dike+EWS', 'Policy 3: RfR+EWS','Policy 7: Dike+RfR+EWS'])]
exp_select.reset_index(inplace=True)
out_select = df.loc[exp.policy.isin(['Policy 5: Dike+EWS', 'Policy 3: RfR+EWS','Policy 7: Dike+RfR+EWS'])]
out_select.reset_index(inplace=True)
x= exp_select[[
       'A.0_ID flood wave shape', 'A.1_Bmax', 'A.1_Brate', 'A.1_pfail',
       'A.2_Bmax', 'A.2_Brate', 'A.2_pfail', 'A.3_Bmax', 'A.3_Brate',
       'A.3_pfail', 'A.4_Bmax', 'A.4_Brate', 'A.4_pfail', 'A.5_Bmax',
       'A.5_Brate', 'A.5_pfail', 'discount rate 0', 'discount rate 1', 'policy'
       ]]
y1=out_select['A.3_Expected Number of Deaths'] >0
y2=out_select['A.3 Total Costs'] > 250e6
y= np.logical_or(y1,y2)



In [ ]:
print(df.policy.unique())

In [ ]:


#ik heb peel-alpha weggehaald, maar dat is misschien niet helemaal correct?
# put y1 or y2 if you want them seperately: prim_alg = prim.Prim(x, y, threshold=0.7)

prim_alg = prim.Prim(x, y, threshold=0.7)
box1 = prim_alg.find_box()

In [ ]:
box1.show_tradeoff()
plt.show()

In [ ]:
box1.inspect_tradeoff()

In [ ]:
box1.show_pairs_scatter()
plt.show()

In [ ]:
box1.inspect(15, style='graph')
plt.show()

In [ ]:
dimensional_stacking.create_pivot_plot(x, y)
plt.show()